In [ ]:
import os

import pandas as pd
import numpy as np

import scanpy as sc
import quicat
from scipy import sparse as sp
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
reports_dir = '/home/daniele/Code/github_synced/barcoding/quicat_paper_code/reports/'
dpi = 300

#### gep processing

In [ ]:
from scipy.stats import median_abs_deviation
def is_outlier(adata, metric: str, nmads: int):
    M = adata.obs[metric]
    outlier = (M < np.median(M) - nmads * median_abs_deviation(M)) | (
        np.median(M) + nmads * median_abs_deviation(M) < M
    )
    return outlier

In [ ]:
adata = sc.read_10x_h5('/mnt/storage/Daniele/clonal_toolkit_data/single-cell/flex_procodes/A40_01/sample_filtered_feature_bc_matrix.h5')
adata.var_names_make_unique()
adata.var_names = list(adata.var_names[:-2]) + ['Egfp','Mcherry']

In [ ]:
sc.pp.calculate_qc_metrics(adata, inplace=True)

In [ ]:
adata.obs["log10_total_counts"] = np.log10(adata.obs["total_counts"])
adata.obs["log10_n_genes_by_counts"] = np.log10(adata.obs["n_genes_by_counts"])

In [ ]:
adata.obs["outlier"] = (
    is_outlier(adata, "total_counts", 2)
    | is_outlier(adata, "n_genes_by_counts", 2)
    | is_outlier(adata, "pct_counts_in_top_50_genes", 2)
)
adata.obs.outlier.value_counts()

In [ ]:
sc.pl.scatter(adata, x = 'log1p_total_counts', y = 'log1p_n_genes_by_counts', color = 'outlier')

In [ ]:
adata = adata[~adata.obs.outlier].copy()

In [ ]:
adata.layers['raw_counts'] = adata.X.copy()
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, flavor = 'seurat')
sc.pp.pca(adata, mask_var="highly_variable")
sc.pp.neighbors(adata)
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution=.2)

In [ ]:
sc.tl.leiden(adata, resolution=.25)

#### barcodes processing

In [ ]:
clone_mapping = {
    'clone_1': ['Au1_link', 'C_link', 'Ollas_cherry'],
    'clone_2': ['Flag_link', 'Ha_cherry'],
    'clone_3': ['Flag_link', 'S_link', 'VSVg_cherry'],
    'clone_4': ['Flag_link','Ha_link','Nws_cherry'],
    'clone_5' : ['S_link','V5_cherry']
}

In [ ]:
quicat_adata = quicat.read_sc('/mnt/storage/Daniele/quicat_benchmark/single-cell/flex_procodes/bam/quicat/barcodes_output.csv')

In [ ]:
quicat_adata = quicat.tl.assign_clones(quicat_adata, clone_mapping, threshold=.75)

In [ ]:
quicat.pl.barplot(
    quicat_adata, 
    groupby = 'n_barcodes_by_counts', 
    color = '#DDCC77',
    xlabel = 'barcodes per cell', 
    edgecolor='black',  
    dpi=dpi,
    save = f'{reports_dir}figures/fig3/barplot_nr_of_barcode.pdf'
)

In [ ]:
sc.pp.normalize_total(quicat_adata)
sc.pp.log1p(quicat_adata)

#### merge adatas

In [ ]:
adata.obs['n_barcodes_by_counts'] = quicat_adata.obs['n_barcodes_by_counts'].fillna(0)
adata.obs['clone'] = quicat_adata.obs['clone'].astype('category').cat.add_categories('Non-barcoded cells')
adata.obs['clone'] = adata.obs['clone'].fillna('Non-barcoded cells')
adata = adata[adata.obs['clone'] != 'Unassigned'].copy()

#### Figures

In [ ]:
random_indices = np.random.permutation(list(range(adata.shape[0])))

In [ ]:
clone_palette = sns.color_palette('colorblind', n_colors=len(adata.obs['clone'].unique()))

In [ ]:
quicat_adata_filtered = quicat_adata[quicat_adata.obs['clone'] != 'Unassigned'].copy()

In [ ]:
quicat_adata_filtered.uns['clone_colors'] = clone_palette

In [ ]:
sc.pp.pca(quicat_adata_filtered)

In [ ]:
sc.pl.pca(quicat_adata_filtered, color = 'clone', palette = clone_palette)

In [ ]:
sc.pl.umap(adata, color = ['clone'], palette = clone_palette, na_color = 'white')

In [ ]:
ax = sc.pl.heatmap(
    quicat_adata_filtered, 
    groupby='clone', 
    var_names=quicat_adata.var_names, 
    cmap='cividis', 
    vmax=5,
    figsize=(4, 6),
    show=False,
)
plt.title('')
plt.grid(False)


plt.savefig(f'{reports_dir}figures/fig3/heatmap_procodes.pdf', dpi = dpi, bbox_inches='tight')

In [ ]:
ax = sc.pl.dotplot(
    quicat_adata_filtered, 
    groupby='clone', 
    var_names=quicat_adata.var_names, 
    cmap='cividis', 
    vmax=4,
    figsize=(6, 3),
    show=False
)
plt.title('')
plt.grid(False)
plt.savefig(f'{reports_dir}figures/fig3/dotplot_procodes.pdf', dpi = dpi, bbox_inches='tight')

In [ ]:
sc.pl.umap(adata, color = ['clone'], palette = clone_palette, na_color = 'white')

In [ ]:
quicat.pl.stacked_barplot(
    adata=adata,
    groupby='leiden',
    obs_key='clone',
    figsize=(12, 8),
    xlabel='Leiden Cluster',
    ylabel='Cell Type Distribution (%)',
    title='',
    palette = clone_palette,
    edgecolor='black',        
    linewidth=1.5,  
    dpi=dpi,
    save = f'{reports_dir}figures/fig3/stacked_barplot_leiden_by_clone.pdf'         
)

In [ ]:
sc.pl.umap(adata[random_indices, :], 
           color=['clone'],
           groups = [f'clone_{i+1}' for i in range(5)] + ['No barcodes'],
           na_in_legend=False,
           size=50, 
           edgecolor='k', 
           linewidth=0.2, 
           alpha=1, 
           #title='UMAP of Samples', 
           #legend_loc='on data',
           frameon = False,
           legend_fontsize=8, 
           legend_fontweight='bold',
           show = False,
           vmax = 2,
        )

plt.title('')
plt.grid(False)
plt.savefig(f'{reports_dir}figures/fig3/umap_clone_only_barcoded_cells.pdf', dpi = dpi, bbox_inches='tight')

In [ ]:
sc.pl.umap(adata[random_indices, :], 
           color=['clone'],
           na_in_legend=False,
           size=50, 
           edgecolor='k', 
           linewidth=0.2, 
           alpha=1, 
           #title='UMAP of Samples', 
           #legend_loc='on data',
           frameon = False,
           legend_fontsize=8, 
           legend_fontweight='bold',
           show = False,
           vmax = 2,
        )

plt.title('')
plt.grid(False)
plt.savefig(f'{reports_dir}figures/fig3/umap_clone.pdf', dpi = dpi, bbox_inches='tight')

In [ ]:
sc.pl.umap(adata[random_indices, :], 
           color=['leiden'],
           na_in_legend=False,
           size=50, 
           edgecolor='k', 
           linewidth=0.2, 
           alpha=1, 
           #title='UMAP of Samples', 
           #legend_loc='on data',
           frameon = False,
           legend_fontsize=8, 
           legend_fontweight='bold',
           show = False,
           palette='tab20',
        )

plt.title('')
plt.grid(False)
plt.savefig(f'{reports_dir}figures/fig3/umap_leiden.pdf', dpi = dpi, bbox_inches='tight')

In [ ]:
sc.pl.umap(adata[random_indices, :], 
           color=['Mcherry'],
           na_in_legend=False,
           size=50, 
           edgecolor='k', 
           linewidth=0.2, 
           alpha=1, 
           #title='UMAP of Samples', 
           #legend_loc='on data',
           frameon = False,
           legend_fontsize=8, 
           legend_fontweight='bold',
           show = False,
           vmax = 7,
           cmap = 'cividis'
           
        )

plt.title('')
plt.grid(False)
plt.savefig(f'{reports_dir}figures/fig3/umap_mCherry.pdf', dpi = dpi, bbox_inches='tight')

In [ ]:
df_procodes = quicat_adata.to_df()
for procode_tag in df_procodes.columns:
    adata.obs[procode_tag] = df_procodes[procode_tag].fillna(0)
    adata.obs[procode_tag] = adata.obs[procode_tag].fillna(0)
    

In [ ]:
for procode_tag in df_procodes.columns:
    sc.pl.umap(adata, 
           color=procode_tag,
           na_in_legend=False,
           size=50, 
           edgecolor='k', 
           linewidth=0.2, 
           alpha=1, 

           frameon = False,
           legend_fontsize=8, 
           legend_fontweight='bold',
           show = False,
           cmap = 'cividis'
        )
    plt.title('')
    plt.grid(False)
    plt.savefig(f'{reports_dir}figures/fig3/umap_{procode_tag}.pdf', dpi = dpi, bbox_inches='tight')

In [ ]:
clone_1 = adata[(adata.obs.clone == 'clone_1') & (adata.obs.leiden.isin(['0','8']))].copy()

In [ ]:
sc.tl.rank_genes_groups(clone_1, groupby='leiden')

In [ ]:
sc.pl.rank_genes_groups_dotplot(clone_1, values_to_plot = 'logfoldchanges', vmin = -2.5, vmax = 2.5, cmap = 'coolwarm', n_genes = 10, show = False)
plt.savefig(f'{reports_dir}figures/fig3/dotplot_leiden_dge_clone_1_zoom_in.pdf', dpi = dpi, bbox_inches='tight')

In [ ]:
sc.tl.umap(clone_1)

In [ ]:
palette = sns.color_palette('tab20')

In [ ]:
palette = [palette[0], palette[16]]

In [ ]:
sc.pl.umap(clone_1, 
           color=['leiden'],
           na_in_legend=False,
           size=200, 
           edgecolor='k', 
           linewidth=0.2, 
           alpha=1, 
           frameon = False,
           legend_fontsize=8, 
           legend_fontweight='bold',
           show = False,
           palette = palette,           
        )

plt.title('')
plt.grid(False)
plt.savefig(f'{reports_dir}figures/fig3/umap_clone_1_zoom_in.pdf', dpi = dpi, bbox_inches='tight')

In [ ]:
top_genes_leiden_8 = sc.get.rank_genes_groups_df(clone_1, group = '8')['names'].values[:10]

In [ ]:
for gene in top_genes_leiden_8:
    sc.pl.umap(clone_1, 
           color=gene,
           na_in_legend=False,
           size=200, 
           edgecolor='k', 
           linewidth=0.2, 
           alpha=1, 
           frameon = False,
           legend_fontsize=8, 
           legend_fontweight='bold',
           show = False,
           cmap = 'cividis'
        )
    plt.title('')
    plt.grid(False)
    plt.savefig(f'{reports_dir}figures/fig3/umap_clone_1_{gene}.pdf', dpi = dpi, bbox_inches='tight')

In [ ]:
import decoupler as dc

In [ ]:
msigdb = dc.get_resource('MSigDB')


In [ ]:
msigdb = msigdb[msigdb['collection']=='hallmark']
msigdb = msigdb[~msigdb.duplicated(['geneset', 'genesymbol'])]


In [ ]:
msigdb_mouse = dc.translate_net(
    msigdb,
    target_organism='mouse',
)
msigdb_mouse = msigdb_mouse[~msigdb_mouse.duplicated(['geneset', 'genesymbol'])]


In [ ]:
dc.run_ora(
    mat=clone_1,
    net=msigdb_mouse,
    source='geneset',
    target='genesymbol',
    verbose=True,
    use_raw=False
)

In [ ]:
acts = dc.get_acts(clone_1, obsm_key='ora_estimate')
acts_v = acts.X.ravel()
max_e = np.nanmax(acts_v[np.isfinite(acts_v)])
acts.X[~np.isfinite(acts.X)] = max_e

In [ ]:
df = dc.rank_sources_groups(acts, groupby='leiden', reference='rest', method='t-test_overestim_var')
n_markers = 5
source_markers = df.groupby('group').head(n_markers).groupby('group')['names'].apply(lambda x: list(x)).to_dict()

In [ ]:
sc.pl.dotplot(acts, source_markers, 'leiden', dendrogram=False, standard_scale='var',
                 colorbar_title='Z-scaled scores', cmap='coolwarm', figsize=(6,2), show = False)
plt.savefig(f'{reports_dir}figures/fig3/umap_clone_1_enrichment_dotplot.pdf', dpi = dpi, bbox_inches='tight')